In [75]:
import hoomd,imp
import hoomd
from hoomd import *
from hoomd import md
import numpy as np
import gsd.hoomd
import time 

t = time.strftime(time.strftime("%d %b %H:%M:%S", time.gmtime()))
seed=42
deg=30
Angle = deg*np.pi/180.
beadr=0.8
patchr=0.4
br_pr=beadr-patchr
context.initialize("")
# Place the type R central particles
uc = hoomd.lattice.unitcell(N = 1,
                            a1 = [5, 0,   0],
                            a2 = [0,    5, 0],
                            a3 = [0,    0,   5],
                            dimensions = 3,
                            position = [[0,0,0]],
                            type_name = ['B'],
                            mass = [1.0],
                            diameter=[beadr],
                            moment_inertia = [[0,
                                               1/12*1.0*8**2,
                                               1/12*1.0*8**2]],
                            orientation = [[1, 0, 0, 0]]);
system = hoomd.init.create_lattice(unitcell=uc, n=[5,5,5]);
system.particles.types.add('A');



harmonic = hoomd.md.bond.harmonic(name='polymer')
harmonic.bond_coeff.set('polymer', k=330.0, r0=1.0)



# Add constituent particles of type A and create the patches



rigid = hoomd.md.constrain.rigid();
#rigid = hoomd.group.rigid_center();
part = rigid.set_param('B',
                types=['A']*2,
                positions=[(-(br_pr)*np.cos(Angle),br_pr*np.sin(Angle),0),(br_pr*np.cos(Angle),br_pr*np.sin(Angle),0)],
                diameters=[patchr,patchr]);

rigid.create_bodies()
rigid.validate_bodies()
nl = hoomd.md.nlist.cell();


lj = hoomd.md.pair.lj(r_cut=2.0, nlist=nl);
lj.pair_coeff.set('A', 'A', epsilon=1.5, sigma = 1.0);
lj.pair_coeff.set('A', 'B', epsilon=0.5, sigma = 1.0);
lj.pair_coeff.set('B', 'B', epsilon=0.5, sigma = 1.0);



dump.gsd(filename='PP00 %s.gsd'%t,period=250,group=group.all(),phase=0)

hoomd.md.integrate.mode_standard(dt=0.01, aniso=True);
rigid = hoomd.group.rigid_center();
all = hoomd.group.all();

integrator = md.integrate.mode_standard(dt=0.001, aniso=True) 

hoomd.md.integrate.langevin(group=rigid, kT=1.0, seed=42);
hoomd.analyze.log(filename="log-output.log",
                  quantities=['potential_energy', 'temperature'],
                  period=500,
                  overwrite=True);
hoomd.run(5e4);
# langevin.disable()
uc.get_type_list()
uc.get_typeid_mapping()
uc.get_snapshot()


notice(2): Group "all" created containing 125 particles


**ERROR**: No bonds are defined.


RuntimeError: Error creating bond forces

In [65]:
uc.get_snapshot()


In [60]:
uc.get_typeid_mapping()

{'B': 0}

In [61]:
uc.get_type_list()

['B']